In [1]:
import os
import pandas as pd
import numpy as np
import random as rd
import datetime as dt
import re
from datetime import timedelta
import warnings
warnings.filterwarnings('ignore')
import time
import csv

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.width', 1000)

# 1. Headcount Data

## 1.1 Load Headcount Data 

### 1.1.1 Merge each month's data to get FY level data

In [14]:
jun = pd.read_csv('./1. Data/Headcount/JUN HC.csv')
jul = pd.read_csv('./1. Data/Headcount/JUL HC.csv')
aug = pd.read_csv('./1. Data/Headcount/AUG HC.csv')

In [16]:
#append the dfs 
df = jun.append([jul, aug])

In [17]:
print(df.shape)

(567079, 114)


In [32]:
#To get unique values by FY (sorted)
#Sort by date
df_sorted = df.sort_values(by='FISCAL_MONTH_END_DATE', ascending=True)

#drop all the duplicates, keeping only the most recent row of the HC
df_unique = df_sorted.drop_duplicates(subset=['EMAIL_ADDRESS'],  keep='last')


# 2. Change Indicators

## 2.1 Load the change indicators

In [47]:
ind = pd.read_csv('./1. Data/Indicators/Change indicators.csv')

In [54]:
#to get the counts for the indicator columns, use transform and sum

col_list = ['ORG_CHANGE_INDICATOR','CAREER_LEVEL_CHANGE_INDICATOR','SUP_CHANGE_INDICATOR','JOB_CHANGE_INDICATOR',
           'LOCATION_CHANGE_INDICATOR','SALARY_CHANGE_COUNT']


#create a for loop to get all the columns that match the column list keyword
for i in col_list:
    fy_cols = df_fy_ind.filter(like=i).columns
    fy2_cols = df_fy2_ind.filter(like=i).columns
    
    if len(fy_cols) > 0:
        df_fy_ind[f'PFY_NUM_{i}'] = (df_fy_ind[fy_cols].groupby(df_fy_ind['EMAIL_ADDRESS']).transform('sum'))
    
    if len(fy2_cols) > 0:
        df_fy2_ind[f'CFY_NUM_{i}'] = (df_fy2_ind[fy2_cols].groupby(df_fy2_ind['EMAIL_ADDRESS']).transform('sum'))

In [57]:
#drop all the duplicates, keeping only the most recent row of the employee
df_fy_ind2 = df_fy_ind.drop_duplicates(subset=['EMAIL_ADDRESS'],  keep='last')
df_fy2_ind2 = df_fy2_ind.drop_duplicates(subset=['EMAIL_ADDRESS'],  keep='last')


In [59]:
#subset the columns for merging

df_fy_ind3 = df_fy_ind2[['EMAIL_ADDRESS','PFY_NUM_ORG_CHANGE_INDICATOR','PFY_NUM_CAREER_LEVEL_CHANGE_INDICATOR',
                           'PFY_NUM_SUP_CHANGE_INDICATOR','PFY_NUM_JOB_CHANGE_INDICATOR','PFY_NUM_LOCATION_CHANGE_INDICATOR',
                            'PFY_NUM_SALARY_CHANGE_COUNT']]

df_fy2_ind3 = df_fy2_ind2[['EMAIL_ADDRESS','CFY_NUM_ORG_CHANGE_INDICATOR','CFY_NUM_CAREER_LEVEL_CHANGE_INDICATOR',
                           'CFY_NUM_SUP_CHANGE_INDICATOR','CFY_NUM_JOB_CHANGE_INDICATOR','CFY_NUM_LOCATION_CHANGE_INDICATOR',
                            'CFY_NUM_SALARY_CHANGE_COUNT']]

## 2.2 Merge the change indicators

In [68]:
#overall_indicators.head()

In [69]:
HC_withIndicators = pd.merge(HC_unique, ind, on='EMAIL_ADDRESS',how ='left')


In [70]:
HC_withIndicators = pd.merge(HC_withIndicators, ind, on='EMAIL_ADDRESS',how ='left')


In [71]:
HC_withIndicators.shape

(66016, 126)

# 3. Termination data

## 3.1 Load termination data and clean it

In [80]:
#load term data
Term = pd.read_csv ('./1. Data/Attrition/Attrition.csv', encoding='utf-8')


In [84]:
#Select Date Columns
datecols = ['LATEST_HIRE_DATE','CONTINUOUS_SERVICE_DATE', 'TERMINATION_DATE', 'JOB_DATE', 'LAST_SALARY_INCR_DATE']

#Convert all date columns to pandas datetime format
term[datecols] = term[datecols].apply(pd.to_datetime, dayfirst=True)


In [86]:
#Sort by Termination Dates
Termcols_sorted = Termcols.sort_values(by='LATEST_HIRE_DATE', ascending=True)


#drop all the duplicates, keeping only the most recent row of the term
# doing this ensures that only the most recent termination is considered for those rehire cases. Also, each terminated employee should only have one termination 
Termcols_unique = Termcols_sorted.drop_duplicates(subset=['EMAIL_ADDRESS'],  keep='last')

## 3.2 Merge the termination data and the HC data

In [88]:
#merge the HC data with Term data
HC_Term = pd.merge(HC_withIndicators, Termcols_unique, on='EMAIL_ADDRESS',how ='left')

In [95]:
#Add a column of 'Terminated' - to identify Active or Terminated
HC_Term['TERMINATED'] = np.where(HC_Term['TERM_SUB_GROUP_TYPE_DESCRIPTION'].isnull(), 'ACTIVE', 'TERMINATED')

# 4. Survey Data

## 4.1 Load and clean the Survey 1 data

In [99]:
survey1 = pd.read_csv('./1. Data/Q3_clean_processed_datav2.csv')

## 4.2 Merge the survey1 scores with the HC data

In [126]:
survey1merge = pd.merge(survey1, hc, on='MANAGER_EMAIL_ADDRESS',how ='left')


# 5. Attainment

## 5.1 Load and clean the attainment data

In [141]:
#load sales attainment data
df_attn = pd.read_csv('./1. Data/SalesAttainment/Attainment.csv',encoding= 'unicode_escape')

## 5.2 Merge the sales attainment data with the HC data

In [157]:

merged2 = pd.merge(df_attn, survey1merge, on='EMAIL_ADDRESS',how ='left')


# 6. Manager Category Data

## 6.1 Load and merge the manager assignment

In [220]:
df_mgr_ah = pd.read_csv('./1. Data/ManagerCategory/Assignment.csv',encoding= 'utf-8')

In [235]:
merged3 = pd.merge(merged2, df_mgr_ah, on='EMAIL_ADDRESS',how ='left')



# 7. Survey2

## 7.1 Load and clean Survey2 data

In [236]:
#read the data
df_survey2 = pd.read_excel('./1. Data/survey2/survey2.xlsx')

In [ ]:
merged4 = pd.merge(merged3, df_survey2, on='EMAIL_ADDRESS',how ='left')
